# Практикум: Введение в NLP и эмбеддинги (TF‑IDF, Word2Vec)

Этот ноутбук — пошаговый практический разбор простых, но ключевых идей NLP на классическом Python (минимум «магии», больше прозрачных вычислений). Мы:
- загрузим и подготовим простой датасет твитов о катастрофах;
- посчитаем TF‑IDF «вручную» и визуализируем документы с помощью Plotly;
- обучим мини‑версии Word2Vec (CBOW и Skip‑Gram) на `numpy` (с нуля);
- разберём функции активации и потерь и посмотрим на получившиеся эмбеддинги слов.

План:
1) Датасеты и контекст занятия  
2) Загрузка данных (Kaggle Disaster Tweets) + фоллбек  
3) Предобработка текста (очистка, токенизация, стоп‑слова)  
4) TF‑IDF: формулы, реализация на Python, визуализация  
5) Word2Vec: CBOW и Skip‑Gram (numpy), функции потерь и активации  
6) Визуализация эмбеддингов слов + похожесть по косинусу  
7) Выводы и доп.материалы

Зависимости: `python>=3.9`, `numpy`, `pandas`, `plotly`. Ноутбук старается обходиться без внешних NLP‑библиотек и «тяжёлых» фреймворков.

Если какой‑то пакет отсутствует, установите его заранее в окружении:

```bash
pip install numpy pandas plotly
```



## 1) Датасеты и контекст занятия

Для практики подойдёт небольшой, понятный по структуре корпус. В этом ноутбуке предпочтение датасету твитов о катастрофах:
- Kaggle: `Disaster Tweets` — `target` (0/1), `text`, иногда `keyword`, `location`. Ссылка: https://www.kaggle.com/datasets/vstepanenko/disaster-tweets

Также, на будущее (если захочется альтернатив):
- SMS Spam Collection (классификация спам/хам): https://archive.ics.uci.edu/dataset/228/sms+spam+collection  
- 20 Newsgroups (классификация новостных тем): https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset  
- IMDb Reviews (оценка тональности отзывов): http://ai.stanford.edu/~amaas/data/sentiment/  
- AG News (классификация новостей): https://huggingface.co/datasets/ag_news

Мы будем использовать «честный» классический Python с `numpy` и `pandas` (по минимуму) и построим свои вычисления TF‑IDF и Word2Vec без «магических» вызовов. Визуализацию сделаем через Plotly.


In [1]:
# Импорты и базовые настройки
import os, re, math, random
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import plotly.express as px

random.seed(42)
np.random.seed(42)

# Настройки отображения pandas
pd.set_option('display.max_colwidth', 140)



## 2) Загрузка данных (Kaggle Disaster Tweets) + фоллбек

Предпочтительный корпус: Kaggle `Disaster Tweets` (`text`, `target ∈ {0,1}`), ссылка: https://www.kaggle.com/datasets/vstepanenko/disaster-tweets

Чтобы ноутбук запускался «из коробки», предусмотрен фоллбек: если CSV не найден локально, берём небольшой встроенный мини‑набор (20 твитов) со схожей структурой. Для занятия этого достаточно, чтобы пройти весь пайплайн.

Как использовать Kaggle‑датасет локально:
1. Скачайте CSV с Kaggle и положите файл, например, в `data/disaster_tweets.csv` или рядом с ноутбуком как `disaster_tweets.csv`.
2. Перезапустите ячейку ниже — будет использован реальный корпус.



In [2]:
# Поиск CSV и фоллбек на мини‑датасет
candidate_paths = [
    'tweets.csv'
]

csv_path = None
for p in candidate_paths:
    if os.path.exists(p):
        csv_path = p
        break

if csv_path is not None:
    df = pd.read_csv(csv_path)
    # Приводим к ожидаемым столбцам, если имена отличаются
    cols = [c.lower() for c in df.columns]
    df.columns = cols
    if 'text' not in df.columns:
        # Ищем похожие варианты
        for c in df.columns:
            if 'tweet' in c or 'content' in c or 'message' in c:
                df['text'] = df[c]
                break
    if 'target' not in df.columns:
        # Иногда метка может называться иначе
        for c in df.columns:
            if c in ['label', 'is_real', 'is_disaster', 'sentiment']:
                df['target'] = df[c]
                break
    df = df[['text', 'target']].dropna()
else:
    # Мини‑корпус: 20 твитов (синтетический, но близкий по тематике) — 0/1 = не катастрофа/катастрофа
    mini_data = [
        {"text": "Huge fire downtown, buildings evacuated and roads closed", "target": 1},
        {"text": "What a beautiful sunny day at the park!", "target": 0},
        {"text": "Earthquake reported near the coast, people are safe so far", "target": 1},
        {"text": "New cafe opened next to my office, great coffee", "target": 0},
        {"text": "Flood warnings in the city, stay away from low areas", "target": 1},
        {"text": "Just finished a great workout, feeling awesome", "target": 0},
        {"text": "Wildfire spreading due to strong winds, firefighters on site", "target": 1},
        {"text": "Watching a movie tonight with friends", "target": 0},
        {"text": "Bridge collapsed after heavy rain, emergency crews arriving", "target": 1},
        {"text": "My cat learned a new trick today", "target": 0},
        {"text": "Tornado touched down near the highway, take shelter immediately", "target": 1},
        {"text": "Trying a new recipe for dinner", "target": 0},
        {"text": "Explosion heard at chemical plant, area cordoned off", "target": 1},
        {"text": "Best concert ever last night!", "target": 0},
        {"text": "Volcano ash delays flights in the region", "target": 1},
        {"text": "Reading a book on my balcony", "target": 0},
        {"text": "Severe storm expected tonight, charge your phones", "target": 1},
        {"text": "Bought new headphones, love the sound", "target": 0},
        {"text": "Landslide blocks mountain road, rescue teams deployed", "target": 1},
        {"text": "Morning coffee is the best ritual", "target": 0},
    ]
    df = pd.DataFrame(mini_data)

print(df.shape)
df.head(8)


(11370, 2)


,text,target
0,"Communal violence in Bhainsa, Telangana. ""Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…",1
1,"Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…",1
2,Arsonist sets cars ablaze at dealership https://t.co/gOQvyJbpVI,1
3,Arsonist sets cars ablaze at dealership https://t.co/0gL7NUCPlb https://t.co/u1CcBhOWh9,1
4,"""Lord Jesus, your love brings freedom and pardon. Fill me with your Holy Spirit and set my heart ablaze with your l… https://t.co/VlTznn...",0
5,"If this child was Chinese, this tweet would have gone viral. Social media would be ablaze. SNL would have made a racist j…",0
6,"Several houses have been set ablaze in Ngemsibaa village, Oku sub division in the North West Region of Cameroon by… https://t.co/99uHGAzxy2",1
7,Asansol: A BJP office in Salanpur village was set ablaze last night. BJP has alleged that TMC is behind the incident. Police has b…,1


## 3) Предобработка текста: очистка, токенизация, стоп‑слова

Минимальная, понятная предобработка для твитов:
- привести к нижнему регистру;
- удалить URL, упоминания `@user`, хэштеги `#tag` (оставим слово без `#`), пунктуацию и цифры;
- токенизировать простым `split()` по пробелам;
- убрать короткие токены и стоп‑слова.

Стоп‑слова возьмём небольшим встроенным набором, чтобы не тянуть NLTK/списки из интернета. Это учебный пример; в реальных задачах список лучше расширить.



In [3]:
# Небольшой список английских стоп‑слов (можно дополнять при желании)
stopwords_en = {
    'a','an','the','and','or','but','if','while','with','without','of','to','in','on','at','for','from','by','about','as',
    'is','are','was','were','be','been','being','am','do','does','did','doing','have','has','had','having',
    'i','you','he','she','it','we','they','me','him','her','them','my','your','his','its','our','their','this','that','these','those',
    'so','very','just','not','no','nor','too','can','cannot','could','should','would','will','shall','may','might',
    'up','down','out','over','under','again','further','then','once',
}

# Простая токенизация/очистка без внешних библиотек
url_re = re.compile(r'https?://\S+|www\.\S+')
mention_re = re.compile(r'@\w+')
hashtag_re = re.compile(r'#')
nonalpha_re = re.compile(r'[^a-z\s]')

# Функции компактные, чтобы было понятно из кода, что происходит

def clean_text_to_tokens(text):
    text = str(text).lower()
    text = url_re.sub(' ', text)
    text = mention_re.sub(' ', text)
    text = hashtag_re.sub('', text)  # оставляем слово без символа #
    text = nonalpha_re.sub(' ', text)
    tokens = [t for t in text.split() if t and t.isalpha()]
    tokens = [t for t in tokens if len(t) > 2 and t not in stopwords_en]
    return tokens

# Применяем к корпусу
df['tokens'] = [clean_text_to_tokens(t) for t in df['text']]

df[['text','tokens','target']].head(6)


,text,tokens,target
0,"Communal violence in Bhainsa, Telangana. ""Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…","[communal, violence, bhainsa, telangana, stones, pelted, muslims, houses, some, houses, vehicles, set, ablaze]",1
1,"Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…","[telangana, section, imposed, bhainsa, january, after, clash, erupted, between, two, groups, january]",1
2,Arsonist sets cars ablaze at dealership https://t.co/gOQvyJbpVI,"[arsonist, sets, cars, ablaze, dealership]",1
3,Arsonist sets cars ablaze at dealership https://t.co/0gL7NUCPlb https://t.co/u1CcBhOWh9,"[arsonist, sets, cars, ablaze, dealership]",1
4,"""Lord Jesus, your love brings freedom and pardon. Fill me with your Holy Spirit and set my heart ablaze with your l… https://t.co/VlTznn...","[lord, jesus, love, brings, freedom, pardon, fill, holy, spirit, set, heart, ablaze]",0
5,"If this child was Chinese, this tweet would have gone viral. Social media would be ablaze. SNL would have made a racist j…","[child, chinese, tweet, gone, viral, social, media, ablaze, snl, made, racist]",0


## 4) TF‑IDF: формулы и реализация «вручную»

Идея: вес слова в документе выше, если оно часто встречается в этом документе (TF), но ниже, если слово часто встречается во всём корпусе (IDF).

- TF(t, d) = count(t, d) / |d|  
- IDF(t, D) = ln((N + 1) / (df(t) + 1)) + 1  — сглажённая версия, где N — число документов, df(t) — в скольких документах встречается t.  
- TF‑IDF(t, d) = TF(t,d) * IDF(t,D)

Материалы:
- Habr: «Краткий обзор техник векторизации в NLP»: https://habr.com/ru/articles/778048/  
- sklearn TfidfVectorizer (для справки): https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html



In [4]:
# Строим словарь корпуса и частоты
N_docs = len(df)
term_doc_freq = Counter()   # df(t)
term_total_freq = Counter() # для отбора наиболее информативных слов

doc_term_counts = []  # списки Counter по документам
for toks in df['tokens']:
    c = Counter(toks)
    doc_term_counts.append(c)
    term_total_freq.update(c)
    term_doc_freq.update(set(c.keys()))

# Ограничим размер словаря, чтобы визуализация и обучение были лёгкими
max_vocab = 2000
vocab_terms = [t for t, _ in term_total_freq.most_common(max_vocab)]
term_to_idx = {t:i for i,t in enumerate(vocab_terms)}

# Считаем IDF по сглажённой формуле
idf = np.zeros(len(vocab_terms), dtype=np.float32)
for t, idx in term_to_idx.items():
    df_t = term_doc_freq.get(t, 0)
    idf[idx] = math.log((N_docs + 1) / (df_t + 1)) + 1.0

# Строим матрицу TF‑IDF (docs x vocab)
X = np.zeros((N_docs, len(vocab_terms)), dtype=np.float32)
for i, c in enumerate(doc_term_counts):
    doc_len = sum(c.values()) if sum(c.values()) > 0 else 1
    for t, cnt in c.items():
        if t in term_to_idx:
            j = term_to_idx[t]
            tf = cnt / doc_len
            X[i, j] = tf * idf[j]

X.shape


(11370, 2000)

In [5]:
# PCA на numpy (через SVD) для визуализации документов TF‑IDF
# Центрируем матрицу и берём первые 2 компоненты
Xc = X - X.mean(axis=0, keepdims=True)
U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
X2d = (U[:, :2] * S[:2])

fig = px.scatter(
    x=X2d[:,0], y=X2d[:,1],
    color=df['target'].astype(str),
    title='Документы в пространстве TF‑IDF (PCA 2D)',
    labels={'color':'target'}
)
fig.show()


## 5) Word2Vec: интуиция, активации и функция потерь

Word2Vec учится предсказывать слово по контексту (CBOW) или контекст по слову (Skip‑Gram). В обоих случаях внутри — два матрицы весов: `W_in` (эмбеддинги «входных» слов) и `W_out` (веса для предсказания слов). На выходе — распределение по словарю через softmax.

- Активация: `softmax(z)` превращает логиты в вероятности:  
  softmax(z)_i = exp(z_i − max(z)) / sum_j exp(z_j − max(z))  
- Потери: кросс‑энтропия с «one‑hot» целевым словом y:  
  L = −log P(y|x) = −log softmax(z)_y

CBOW (контекст → целевое слово):  
- Считаем средний вектор контекста v_c как среднее эмбеддингов контекстных слов из `W_in`.  
- Логиты: z = W_out · v_c  
- Вероятности: p = softmax(z)  
- Градиенты: ∂L/∂z = p − one_hot(y), далее получаем ∂L/∂W_out и ∂L/∂v_c, раскладываем ∂L/∂W_in по словам контекста.

Skip‑Gram (слово → контекстные слова):  
- Берём эмбеддинг центра v_center = W_in[word].  
- Для каждого контекстного слова y делаем предсказание через softmax(W_out · v_center) и считаем кросс‑энтропию.

Первые версии Word2Vec также используют ускорения вроде негативной выборки или иерархического softmax (см. оригинальные статьи):  
- Mikolov et al., Efficient Estimation of Word Representations in Vector Space: https://arxiv.org/abs/1301.3781  
- Mikolov et al., Distributed Representations of Words and Phrases: https://arxiv.org/abs/1310.4546  
- Документация Gensim Word2Vec (для справки): https://radimrehurek.com/gensim/models/word2vec.html



In [6]:
# Готовим подкорпус и словарь для Word2Vec
# Берём лишь часть твитов и ограничим словарь частотными словами, чтобы обучение было быстрым
sentences = [toks for toks in df['tokens'] if len(toks) > 0]
max_sentences = min(len(sentences), 400)
sentences = sentences[:max_sentences]

# Частоты по подкорпусу
freq = Counter()
for s in sentences:
    freq.update(s)

vocab_size_limit = 600
most_common = [w for w, _ in freq.most_common(vocab_size_limit)]
word_to_idx = {w:i for i,w in enumerate(most_common)}
idx_to_word = {i:w for w,i in word_to_idx.items()}

# Строим обучающие пары для CBOW: (контекст -> центр)
window = 2
cbow_pairs = []
for s in sentences:
    idxs = [word_to_idx[w] for w in s if w in word_to_idx]
    for i, center in enumerate(idxs):
        left = max(0, i - window)
        right = min(len(idxs), i + window + 1)
        ctx = [idxs[j] for j in range(left, right) if j != i]
        if len(ctx) > 0:
            cbow_pairs.append((ctx, center))

# Чтобы ускорить, случайно подвыберем пары
random.shuffle(cbow_pairs)
cbow_pairs = cbow_pairs[:2000]

len(sentences), len(word_to_idx), len(cbow_pairs)


(400, 600, 2000)

In [7]:
# Обучение CBOW (numpy, полный softmax и кросс‑энтропия)
# Минимальная реализация для учебных целей — небольшие размеры, несколько эпох

def softmax(z):
    z = z - np.max(z)
    e = np.exp(z)
    return e / (np.sum(e) + 1e-12)

vocab_size = len(word_to_idx)
embed_dim = 30
lr = 0.05
epochs = 100

W_in = 0.01 * np.random.randn(vocab_size, embed_dim).astype(np.float32)
W_out = 0.01 * np.random.randn(vocab_size, embed_dim).astype(np.float32)

for ep in range(epochs):
    total_loss = 0.0
    for ctx, center in cbow_pairs:
        # Средний вектор контекста
        v_c = np.mean(W_in[ctx], axis=0)
        # Логиты и вероятности
        logits = W_out @ v_c
        probs = softmax(logits)
        # Потери: -log p(center)
        loss = -np.log(probs[center] + 1e-12)
        total_loss += loss
        
        # Градиенты
        grad_logits = probs
        grad_logits[center] -= 1.0  # p - one_hot
        # dL/dW_out = outer(grad_logits, v_c)
        W_out -= lr * np.outer(grad_logits, v_c)
        # dL/dv_c = W_out^T @ grad_logits
        grad_vc = W_out.T @ grad_logits
        # Раскидываем по словам контекста (среднее)
        grad_each = grad_vc / len(ctx)
        for ci in ctx:
            W_in[ci] -= lr * grad_each
    print(f"[CBOW] epoch {ep+1}/{epochs}, loss={total_loss/len(cbow_pairs):.4f}")

# Эмбеддинги слов — строки W_in
embeddings_cbow = W_in.copy()


[CBOW] epoch 1/100, loss=6.3969
[CBOW] epoch 2/100, loss=6.3966
[CBOW] epoch 3/100, loss=6.3962
[CBOW] epoch 4/100, loss=6.3952
[CBOW] epoch 5/100, loss=6.3921
[CBOW] epoch 6/100, loss=6.3817
[CBOW] epoch 7/100, loss=6.3471
[CBOW] epoch 8/100, loss=6.2853
[CBOW] epoch 9/100, loss=6.2282
[CBOW] epoch 10/100, loss=6.1770
[CBOW] epoch 11/100, loss=6.1200
[CBOW] epoch 12/100, loss=6.0503
[CBOW] epoch 13/100, loss=5.9769
[CBOW] epoch 14/100, loss=5.9112
[CBOW] epoch 15/100, loss=5.8455
[CBOW] epoch 16/100, loss=5.7706
[CBOW] epoch 17/100, loss=5.6801
[CBOW] epoch 18/100, loss=5.5726
[CBOW] epoch 19/100, loss=5.4547
[CBOW] epoch 20/100, loss=5.3351
[CBOW] epoch 21/100, loss=5.2168
[CBOW] epoch 22/100, loss=5.0975
[CBOW] epoch 23/100, loss=4.9761
[CBOW] epoch 24/100, loss=4.8537
[CBOW] epoch 25/100, loss=4.7313
[CBOW] epoch 26/100, loss=4.6082
[CBOW] epoch 27/100, loss=4.4845
[CBOW] epoch 28/100, loss=4.3602
[CBOW] epoch 29/100, loss=4.2356
[CBOW] epoch 30/100, loss=4.1107
[CBOW] epoch 31/100

In [8]:
# Строим пары для Skip‑Gram: (центр -> каждый контекст)
skipgram_pairs = []
for s in sentences:
    idxs = [word_to_idx[w] for w in s if w in word_to_idx]
    for i, center in enumerate(idxs):
        left = max(0, i - window)
        right = min(len(idxs), i + window + 1)
        ctxs = [idxs[j] for j in range(left, right) if j != i]
        for y in ctxs:
            skipgram_pairs.append((center, y))

random.shuffle(skipgram_pairs)
skipgram_pairs = skipgram_pairs[:2000]

# Обучение Skip‑Gram (numpy, тот же softmax + кросс‑энтропия)
W_in_sg = 0.01 * np.random.randn(vocab_size, embed_dim).astype(np.float32)
W_out_sg = 0.01 * np.random.randn(vocab_size, embed_dim).astype(np.float32)

for ep in range(epochs):
    total_loss = 0.0
    for center, y in skipgram_pairs:
        v_center = W_in_sg[center]
        logits = W_out_sg @ v_center
        probs = softmax(logits)
        loss = -np.log(probs[y] + 1e-12)
        total_loss += loss

        grad_logits = probs
        grad_logits[y] -= 1.0
        W_out_sg -= lr * np.outer(grad_logits, v_center)
        grad_vc = W_out_sg.T @ grad_logits
        W_in_sg[center] -= lr * grad_vc
    print(f"[SkipGram] epoch {ep+1}/{epochs}, loss={total_loss/len(skipgram_pairs):.4f}")

embeddings_sg = W_in_sg.copy()


[SkipGram] epoch 1/100, loss=6.3969
[SkipGram] epoch 2/100, loss=6.3964
[SkipGram] epoch 3/100, loss=6.3955
[SkipGram] epoch 4/100, loss=6.3930
[SkipGram] epoch 5/100, loss=6.3846
[SkipGram] epoch 6/100, loss=6.3571
[SkipGram] epoch 7/100, loss=6.3183
[SkipGram] epoch 8/100, loss=6.2700
[SkipGram] epoch 9/100, loss=6.2106
[SkipGram] epoch 10/100, loss=6.1304
[SkipGram] epoch 11/100, loss=6.0395
[SkipGram] epoch 12/100, loss=5.9338
[SkipGram] epoch 13/100, loss=5.8067
[SkipGram] epoch 14/100, loss=5.6640
[SkipGram] epoch 15/100, loss=5.5040
[SkipGram] epoch 16/100, loss=5.3315
[SkipGram] epoch 17/100, loss=5.1543
[SkipGram] epoch 18/100, loss=4.9757
[SkipGram] epoch 19/100, loss=4.7993
[SkipGram] epoch 20/100, loss=4.6258
[SkipGram] epoch 21/100, loss=4.4550
[SkipGram] epoch 22/100, loss=4.2884
[SkipGram] epoch 23/100, loss=4.1269
[SkipGram] epoch 24/100, loss=3.9708
[SkipGram] epoch 25/100, loss=3.8238
[SkipGram] epoch 26/100, loss=3.6953
[SkipGram] epoch 27/100, loss=3.5954
[SkipGram]

### Визуализация словарных эмбеддингов (CBOW и Skip‑Gram)
Снизим размерность эмбеддингов до 2D через PCA (на `numpy`) и отобразим слова. Покажем топ‑N частотных слов, чтобы подписи были читабельными.


In [9]:
# Берём топ частотных слов для подписей
N_show = 120
show_words = [w for w, _ in freq.most_common(N_show) if w in word_to_idx]
show_idx = np.array([word_to_idx[w] for w in show_words], dtype=int)

# PCA на numpy для CBOW
E = embeddings_cbow
Ec = E - E.mean(axis=0, keepdims=True)
Uc, Sc, Vtc = np.linalg.svd(Ec, full_matrices=False)
E2d_cbow = (Uc[:, :2] * Sc[:2])

# PCA для Skip‑Gram
Esg = embeddings_sg
Esgc = Esg - Esg.mean(axis=0, keepdims=True)
Usg, Ssg, Vtsg = np.linalg.svd(Esgc, full_matrices=False)
E2d_sg = (Usg[:, :2] * Ssg[:2])

# Визуализируем CBOW
fig_cbow = px.scatter(x=E2d_cbow[show_idx,0], y=E2d_cbow[show_idx,1], text=show_words,
                      title='Слова в пространстве эмбеддингов (CBOW)')
fig_cbow.update_traces(textposition='top center')
fig_cbow.show()

# Визуализируем Skip‑Gram
fig_sg = px.scatter(x=E2d_sg[show_idx,0], y=E2d_sg[show_idx,1], text=show_words,
                    title='Слова в пространстве эмбеддингов (Skip‑Gram)')
fig_sg.update_traces(textposition='top center')
fig_sg.show()


In [10]:
# Быстрый поиск похожих слов по косинусной близости

def cosine_sim(a, b):
    na = np.linalg.norm(a) + 1e-12
    nb = np.linalg.norm(b) + 1e-12
    return float(np.dot(a, b) / (na * nb))

# Выберем несколько интересных слов, если они есть в словаре
probe_words = ['fire','earthquake','flood','storm','help','safe']

for model_name, E in [('CBOW', embeddings_cbow), ('SkipGram', embeddings_sg)]:
    print(f"\n=== {model_name} похожие слова ===")
    for w in probe_words:
        if w in word_to_idx:
            wi = word_to_idx[w]
            v = E[wi]
            sims = []
            for i in range(E.shape[0]):
                if i == wi: 
                    continue
                sims.append((idx_to_word[i], cosine_sim(v, E[i])))
            sims.sort(key=lambda x: x[1], reverse=True)
            print(w, '→', ', '.join([f"{sw}({s:.2f})" for sw,s in sims[:8]]))
        else:
            print(w, '— нет в словаре')



=== CBOW похожие слова ===
fire → armed(0.63), airport(0.60), homes(0.59), done(0.56), memories(0.53), damage(0.52), vehicles(0.51), event(0.51)
earthquake → dropped(0.63), forecast(0.60), hit(0.56), usgs(0.54), aftershock(0.52), male(0.52), puerto(0.51), within(0.49)
flood → theft(0.88), robbery(0.79), leaving(0.74), armed(0.72), oku(0.53), ambulance(0.52), shameless(0.50), black(0.50)
storm — нет в словаре
help → set(0.60), wish(0.59), usgs(0.57), please(0.57), money(0.56), qassemsoleimani(0.53), team(0.51), forecast(0.51)
safe — нет в словаре

=== SkipGram похожие слова ===
fire → flood(0.98), marilyn(0.98), propaganda(0.98), jan(0.98), dealership(0.98), showing(0.98), through(0.97), everything(0.97)
earthquake → video(0.71), hit(0.71), needs(0.70), outside(0.69), days(0.68), special(0.68), day(0.68), track(0.68)
flood → showing(0.99), feel(0.99), build(0.99), everything(0.99), overbought(0.99), propaganda(0.99), through(0.99), marivan(0.99)
storm — нет в словаре
help → province(0.

## 6) Выводы и что дальше

- **TF‑IDF** даёт разреженные векторы документов, хорошо работает как прочная «база», но не учитывает семантику (синонимы/контекст).  
- **Word2Vec** (CBOW/Skip‑Gram) обучает плотные векторы слов, где близость отражает смысловую близость. Мы явно посчитали softmax и кросс‑энтропию, увидели обновления весов.  
- Визуализация (PCA → Plotly) помогает интуитивно увидеть кластера документов и группировки слов.

Идеи для расширения:
- добавить стемминг/лемматизацию, более полный список стоп‑слов;  
- протестировать разные окна контекста и размер эмбеддингов;  
- попробовать негативную выборку (negative sampling) для ускорения обучения на большем корпусе;  
- вместо PCA взять t‑SNE/UMAP (для больших данных — осторожнее с временем/ресурсами).

Полезные ссылки:
- Kaggle `Disaster Tweets`: https://www.kaggle.com/datasets/vstepanenko/disaster-tweets  
- TF‑IDF разбор (Habr): https://habr.com/ru/articles/778048/  
- Документация Plotly: https://plotly.com/python/  
- Word2Vec оригинал: https://arxiv.org/abs/1301.3781 и https://arxiv.org/abs/1310.4546  
- Gensim Word2Vec (для реальных проектов): https://radimrehurek.com/gensim/models/word2vec.html


### Дополнительные эпохи обучения эмбеддингов

Чтобы улучшить качество векторов, дообучим модели CBOW и Skip‑Gram ещё на несколько эпох, используя те же пары. После дообучения пересохраним `embeddings_cbow` и `embeddings_sg`.


In [11]:
# Дообучение: ещё несколько эпох для CBOW и Skip‑Gram
extra_epochs = 6  # добавим к уже пройденным

# CBOW дообучение
for ep in range(extra_epochs):
    total_loss = 0.0
    for ctx, center in cbow_pairs:
        v_c = np.mean(W_in[ctx], axis=0)
        logits = W_out @ v_c
        probs = 1.0 / (1.0 + np.exp(-(logits - np.max(logits))))  # простая безопасная сигмоида для стабильности по компонентам
        # Переход к softmax: нормируем
        probs = np.exp(logits - np.max(logits))
        probs = probs / (np.sum(probs) + 1e-12)
        loss = -np.log(probs[center] + 1e-12)
        total_loss += loss
        grad_logits = probs
        grad_logits[center] -= 1.0
        W_out -= lr * np.outer(grad_logits, v_c)
        grad_vc = W_out.T @ grad_logits
        grad_each = grad_vc / len(ctx)
        for ci in ctx:
            W_in[ci] -= lr * grad_each
    print(f"[CBOW extra] epoch {ep+1}/{extra_epochs}, loss={total_loss/len(cbow_pairs):.4f}")

# Skip‑Gram дообучение
for ep in range(extra_epochs):
    total_loss = 0.0
    for center, y in skipgram_pairs:
        v_center = W_in_sg[center]
        logits = W_out_sg @ v_center
        probs = np.exp(logits - np.max(logits))
        probs = probs / (np.sum(probs) + 1e-12)
        loss = -np.log(probs[y] + 1e-12)
        total_loss += loss
        grad_logits = probs
        grad_logits[y] -= 1.0
        W_out_sg -= lr * np.outer(grad_logits, v_center)
        grad_vc = W_out_sg.T @ grad_logits
        W_in_sg[center] -= lr * grad_vc
    print(f"[SkipGram extra] epoch {ep+1}/{extra_epochs}, loss={total_loss/len(skipgram_pairs):.4f}")

# Обновим эмбеддинги после дообучения
embeddings_cbow = W_in.copy()
embeddings_sg = W_in_sg.copy()


C:\Users\bizy1\AppData\Local\Temp\ipykernel_39540\1536931192.py:10: RuntimeWarning:

overflow encountered in exp



[CBOW extra] epoch 1/6, loss=1.8244
[CBOW extra] epoch 2/6, loss=2.0043
[CBOW extra] epoch 3/6, loss=2.0913
[CBOW extra] epoch 4/6, loss=2.2573
[CBOW extra] epoch 5/6, loss=2.2415
[CBOW extra] epoch 6/6, loss=2.2762
[SkipGram extra] epoch 1/6, loss=22.0278
[SkipGram extra] epoch 2/6, loss=22.6632
[SkipGram extra] epoch 3/6, loss=22.8827
[SkipGram extra] epoch 4/6, loss=23.2299
[SkipGram extra] epoch 5/6, loss=23.6026
[SkipGram extra] epoch 6/6, loss=23.8016


## Дополнение: простая классификация на основе Word2Vec

Идея максимально простая и прозрачная:
- для каждого документа берём средний вектор слов (из обученных эмбеддингов CBOW/Skip‑Gram);
- обучаем логистическую регрессию (1 слой: `sigmoid(XW+b)`) на `numpy` с кросс‑энтропией;
- смотрим точность на отложенной выборке.

Без функций/классов — всё «в лоб», небольшим количеством кода.


In [12]:
# Документные эмбеддинги как среднее словарных векторов (CBOW и Skip‑Gram)
embed_dim = embeddings_cbow.shape[1]
X_doc_cbow = np.zeros((len(df), embed_dim), dtype=np.float32)
X_doc_sg   = np.zeros((len(df), embed_dim), dtype=np.float32)

for i, toks in enumerate(df['tokens']):
    idxs = [word_to_idx[w] for w in toks if w in word_to_idx]
    if len(idxs) > 0:
        X_doc_cbow[i] = embeddings_cbow[idxs].mean(axis=0)
        X_doc_sg[i]   = embeddings_sg[idxs].mean(axis=0)
    else:
        X_doc_cbow[i] = 0.0
        X_doc_sg[i]   = 0.0

y = df['target'].astype(np.float32).values

X_doc_cbow.shape, X_doc_sg.shape, y.shape


((11370, 30), (11370, 30), (11370,))

In [13]:
# Логистическая регрессия на эмбеддингах CBOW (numpy)
perm = np.random.permutation(len(df))
train_size = int(0.8 * len(df))
train_idx, test_idx = perm[:train_size], perm[train_size:]

Xtr = X_doc_cbow[train_idx]
Xte = X_doc_cbow[test_idx]
ytr = y[train_idx]
yte = y[test_idx]

# Стандартизация (по обучающей выборке)
mu = Xtr.mean(axis=0)
sigma = Xtr.std(axis=0) + 1e-8
Xtr = (Xtr - mu) / sigma
Xte = (Xte - mu) / sigma

# Инициализация весов и обучение (полный батч)
W = np.zeros(embed_dim, dtype=np.float32)
b = 0.0
lr = 0.5
epochs = 200

for ep in range(epochs):
    z = Xtr @ W + b
    p = 1.0 / (1.0 + np.exp(-z))
    loss = -np.mean(ytr*np.log(p + 1e-12) + (1 - ytr)*np.log(1 - p + 1e-12))
    grad = (p - ytr)
    dW = Xtr.T @ grad / Xtr.shape[0]
    db = grad.mean()
    W -= lr * dW
    b -= lr * db
    if (ep + 1) % 50 == 0:
        print(f"[CBOW LR] epoch {ep+1}/{epochs}, loss={loss:.4f}")

# Оценка
p_te = 1.0 / (1.0 + np.exp(- (Xte @ W + b)))
pred = (p_te >= 0.5).astype(np.int32)
acc = (pred == yte).mean()

TP = int(((pred == 1) & (yte == 1)).sum())
TN = int(((pred == 0) & (yte == 0)).sum())
FP = int(((pred == 1) & (yte == 0)).sum())
FN = int(((pred == 0) & (yte == 1)).sum())

print(f"CBOW Test accuracy: {acc:.3f} | TP={TP} TN={TN} FP={FP} FN={FN}")


[CBOW LR] epoch 50/200, loss=0.4425
[CBOW LR] epoch 100/200, loss=0.4413
[CBOW LR] epoch 150/200, loss=0.4411
[CBOW LR] epoch 200/200, loss=0.4411
CBOW Test accuracy: 0.798 | TP=10 TN=1805 FP=16 FN=443


In [14]:
# Логистическая регрессия на эмбеддингах Skip‑Gram (numpy) — такой же код
perm = np.random.permutation(len(df))
train_size = int(0.8 * len(df))
train_idx, test_idx = perm[:train_size], perm[train_size:]

Xtr = X_doc_sg[train_idx]
Xte = X_doc_sg[test_idx]
ytr = y[train_idx]
yte = y[test_idx]

mu = Xtr.mean(axis=0)
sigma = Xtr.std(axis=0) + 1e-8
Xtr = (Xtr - mu) / sigma
Xte = (Xte - mu) / sigma

W = np.zeros(embed_dim, dtype=np.float32)
b = 0.0
lr = 0.5
epochs = 200

for ep in range(epochs):
    z = Xtr @ W + b
    p = 1.0 / (1.0 + np.exp(-z))
    loss = -np.mean(ytr*np.log(p + 1e-12) + (1 - ytr)*np.log(1 - p + 1e-12))
    grad = (p - ytr)
    dW = Xtr.T @ grad / Xtr.shape[0]
    db = grad.mean()
    W -= lr * dW
    b -= lr * db
    if (ep + 1) % 50 == 0:
        print(f"[SG LR] epoch {ep+1}/{epochs}, loss={loss:.4f}")

p_te = 1.0 / (1.0 + np.exp(- (Xte @ W + b)))
pred = (p_te >= 0.5).astype(np.int32)
acc = (pred == yte).mean()

TP = int(((pred == 1) & (yte == 1)).sum())
TN = int(((pred == 0) & (yte == 0)).sum())
FP = int(((pred == 1) & (yte == 0)).sum())
FN = int(((pred == 0) & (yte == 1)).sum())

print(f"Skip‑Gram Test accuracy: {acc:.3f} | TP={TP} TN={TN} FP={FP} FN={FN}")


[SG LR] epoch 50/200, loss=0.4842
[SG LR] epoch 100/200, loss=0.4841
[SG LR] epoch 150/200, loss=0.4840
[SG LR] epoch 200/200, loss=0.4839
Skip‑Gram Test accuracy: 0.825 | TP=0 TN=1875 FP=0 FN=399
